In [1]:
%load_ext autoreload
%autoreload 2
import torch as t
import matplotlib.pyplot as plt
from gninvert.functions import sort_with
from gninvert.graph_compare import model_compare
from gninvert.rule_discovery import *
from gninvert.hyperparamsearch import get_hyperparam_dtree, view_hp_results_graph, hp_stats
from gninvert.dtree import decision_tree_paths
from gninvert.gns import RecoveredGN

In [5]:
sr_on_gn_diff1 = t.load('runs/SR_diff1/sr')
sr_on_gn_diff1

{'message_sr_result': complexity                      5
 loss                          0.0
 score                         inf
 equation        ((xs - xt) * 0.1)
 sympy_format      0.1*xs - 0.1*xt
 Name: 2, dtype: object,
 'update_sr_result': complexity             3
 loss                 0.0
 score                inf
 equation        (xt + a)
 sympy_format      a + xt
 Name: 1, dtype: object,
 'message_variable_order': [xt, xs],
 'update_variable_order': [xt, a],
 'data_trained_on': <gninvert.data_generation.TrainingData, 6 train size / 2 validation size / shuffle False>}

In [7]:
sr_on_gn_diff2 = t.load('runs/SR_diff2/sr')
sr_on_gn_diff2

{'message_sr_result': [complexity                                    5
  loss                                        0.0
  score                                       inf
  equation             ((xs1 - xt1) * 0.10000013)
  sympy_format    0.10000013*xs1 - 0.10000013*xt1
  Name: 2, dtype: object,
  complexity                         5
  loss                             0.0
  score                            inf
  equation        ((xt2 - xs2) * -0.1)
  sympy_format       0.1*xs2 - 0.1*xt2
  Name: 2, dtype: object],
 'update_sr_result': [complexity               3
  loss                   0.0
  score                  inf
  equation        (xt1 + a1)
  sympy_format      a1 + xt1
  Name: 1, dtype: object,
  complexity               3
  loss                   0.0
  score                  inf
  equation        (xt2 + a2)
  sympy_format      a2 + xt2
  Name: 1, dtype: object],
 'message_variable_order': [[xt1, xt2, xs1, xs2], [xt1, xt2, xs1, xs2]],
 'update_variable_order': [[xt1, xt2, a1, a2]

In [8]:
sr_on_gn_diff3 = t.load('runs/SR_diff3/sr')
sr_on_gn_diff3

{'message_sr_result': [complexity                         5
  loss                             0.0
  score                            inf
  equation        ((xt1 - xs1) * -0.1)
  sympy_format       0.1*xs1 - 0.1*xt1
  Name: 2, dtype: object,
  complexity                                    5
  loss                                        0.0
  score                                       inf
  equation             ((xs2 - xt2) * 0.09999987)
  sympy_format    0.09999987*xs2 - 0.09999987*xt2
  Name: 2, dtype: object,
  complexity                                      5
  loss                                          0.0
  score                                         inf
  equation             ((xt3 - xs3) * -0.099999994)
  sympy_format    0.099999994*xs3 - 0.099999994*xt3
  Name: 2, dtype: object],
 'update_sr_result': [complexity               3
  loss                   0.0
  score                  inf
  equation        (a1 + xt1)
  sympy_format      a1 + xt1
  Name: 1, dtype: object,
  co

In [11]:
sr_on_gn_actinh = t.load('runs/SR_act_inh_simple/sr')
sr_on_gn_actinh
# NOTE! Below, the a and xt variables are swapped in the update equations
# (this bug, purely about naming, existed when the runs were made but has since been fixed)

{'message_sr_result': [complexity                         5
  loss                             0.0
  score                            inf
  equation        ((xt2 - xs2) * -0.1)
  sympy_format       0.1*xs2 - 0.1*xt2
  Name: 2, dtype: object,
  complexity                                    5
  loss                                        0.0
  score                                       inf
  equation             ((xs3 - xt3) * 0.04999993)
  sympy_format    0.04999993*xs3 - 0.04999993*xt3
  Name: 2, dtype: object],
 'update_sr_result': [complexity                                      9
  loss                                          0.0
  score                                         inf
  equation        (a1 + (((a3 - a2) * -0.05) / a1))
  sympy_format          a1 + (0.05*a2 - 0.05*a3)/a1
  Name: 4, dtype: object,
  complexity               3
  loss                   0.0
  score                  inf
  equation        (xt1 + a2)
  sympy_format      a2 + xt1
  Name: 1, dtype: object,
  co

In [12]:
sr_on_gn_actinh_full = t.load('runs/SR_act_inh_full/sr')
sr_on_gn_actinh_full
# NOTE! Below, the a and xt variables are swapped in the update equations
# (this bug, purely about naming, existed when the runs were made but has since been fixed)

{'message_sr_result': [complexity                                    5
  loss                                        0.0
  score                                       inf
  equation             ((xs2 - xt2) * 0.10000013)
  sympy_format    0.10000013*xs2 - 0.10000013*xt2
  Name: 2, dtype: object,
  complexity                         5
  loss                             0.0
  score                            inf
  equation        ((xs3 - xt3) * 0.01)
  sympy_format     0.01*xs3 - 0.01*xt3
  Name: 2, dtype: object],
 'update_sr_result': [complexity                       3
  loss                      0.000512
  score                     0.366335
  equation        (a1 + 0.023522463)
  sympy_format      a1 + 0.023522463
  Name: 1, dtype: object,
  complexity                                                     11
  loss                                                     0.006075
  score                                                    0.884396
  equation        ((a2 * ((a2 * 0.18499938) / 